<h1>Telecom Customer Churn Prediction</h1> 

<h2>Importing Necessary Libraries</h2>

In [1]:
# Data Manipulation Libraries
import pandas as pd 
import numpy as np

# Plotting Libraries
import seaborn as sns
import matplotlib.pyplot as plt

# Modelling
from scipy.stats import norm
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# Metrics
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, recall_score, confusion_matrix

In [2]:
# Load the dataset

data = pd.read_csv('C:\\Users\\Tsi\\Desktop\\10academy\\Week11\\kaggle\\input\\train.csv',low_memory=False)
df_test = pd.read_csv('C:\\Users\\Tsi\\Desktop\\10academy\\Week11\\kaggle\\input\\test .csv',low_memory=False)

data.head()

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
0,OH,107,area_code_415,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,no
1,NJ,137,area_code_415,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,no
2,OH,84,area_code_408,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,no
3,OK,75,area_code_415,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,no
4,MA,121,area_code_510,no,yes,24,218.2,88,37.09,348.5,108,29.62,212.6,118,9.57,7.5,7,2.03,3,no


In [3]:
df_test.head()

,id,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
0,1,KS,128,area_code_415,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,2,AL,118,area_code_510,yes,no,0,223.4,98,37.98,220.6,101,18.75,203.9,118,9.18,6.3,6,1.70,0
2,3,IA,62,area_code_415,no,no,0,120.7,70,20.52,307.2,76,26.11,203.0,99,9.14,13.1,6,3.54,4
3,4,VT,93,area_code_510,no,no,0,190.7,114,32.42,218.2,111,18.55,129.6,121,5.83,8.1,3,2.19,3
4,5,NE,174,area_code_415,no,no,0,124.3,76,21.13,277.1,112,23.55,250.7,115,11.28,15.5,5,4.19,3


<h2>Understanding the data</h2>

In [8]:
data.shape

(4250, 20)

In [9]:
df_test.shape

(750, 20)

In [10]:
data.columns

Index(['state', 'account_length', 'area_code', 'international_plan',
       'voice_mail_plan', 'number_vmail_messages', 'total_day_minutes',
       'total_day_calls', 'total_day_charge', 'total_eve_minutes',
       'total_eve_calls', 'total_eve_charge', 'total_night_minutes',
       'total_night_calls', 'total_night_charge', 'total_intl_minutes',
       'total_intl_calls', 'total_intl_charge',
       'number_customer_service_calls', 'churn'],
      dtype='object')

In [11]:
df_test.columns

Index(['id', 'state', 'account_length', 'area_code', 'international_plan',
       'voice_mail_plan', 'number_vmail_messages', 'total_day_minutes',
       'total_day_calls', 'total_day_charge', 'total_eve_minutes',
       'total_eve_calls', 'total_eve_charge', 'total_night_minutes',
       'total_night_calls', 'total_night_charge', 'total_intl_minutes',
       'total_intl_calls', 'total_intl_charge',
       'number_customer_service_calls'],
      dtype='object')

In [12]:
data.nunique().sort_values()

churn                               2
international_plan                  2
voice_mail_plan                     2
area_code                           3
number_customer_service_calls      10
total_intl_calls                   21
number_vmail_messages              46
state                              51
total_day_calls                   120
total_eve_calls                   123
total_night_calls                 128
total_intl_minutes                168
total_intl_charge                 168
account_length                    215
total_night_charge                992
total_eve_charge                 1572
total_night_minutes              1757
total_eve_minutes                1773
total_day_charge                 1843
total_day_minutes                1843
dtype: int64

In [13]:
#checking for culomns data type

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4250 entries, 0 to 4249
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   state                          4250 non-null   object 
 1   account_length                 4250 non-null   int64  
 2   area_code                      4250 non-null   object 
 3   international_plan             4250 non-null   object 
 4   voice_mail_plan                4250 non-null   object 
 5   number_vmail_messages          4250 non-null   int64  
 6   total_day_minutes              4250 non-null   float64
 7   total_day_calls                4250 non-null   int64  
 8   total_day_charge               4250 non-null   float64
 9   total_eve_minutes              4250 non-null   float64
 10  total_eve_calls                4250 non-null   int64  
 11  total_eve_charge               4250 non-null   float64
 12  total_night_minutes            4250 non-null   f

In [14]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750 entries, 0 to 749
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             750 non-null    int64  
 1   state                          750 non-null    object 
 2   account_length                 750 non-null    int64  
 3   area_code                      750 non-null    object 
 4   international_plan             750 non-null    object 
 5   voice_mail_plan                750 non-null    object 
 6   number_vmail_messages          750 non-null    int64  
 7   total_day_minutes              750 non-null    float64
 8   total_day_calls                750 non-null    int64  
 9   total_day_charge               750 non-null    float64
 10  total_eve_minutes              750 non-null    float64
 11  total_eve_calls                750 non-null    int64  
 12  total_eve_charge               750 non-null    flo

In [15]:
#fraudresult distribution

Churn_result = data['churn'].value_counts(normalize=True)
print("\nChurnresult distribution:")
print(Churn_result)


Churnresult distribution:
churn
no     0.859294
yes    0.140706
Name: proportion, dtype: float64


In [16]:
#check for missing and duplicated 

print(data.isna().sum().sort_values(ascending = False))
print('duplicate values in df' ,data.duplicated().sum() )

state                            0
account_length                   0
number_customer_service_calls    0
total_intl_charge                0
total_intl_calls                 0
total_intl_minutes               0
total_night_charge               0
total_night_calls                0
total_night_minutes              0
total_eve_charge                 0
total_eve_calls                  0
total_eve_minutes                0
total_day_charge                 0
total_day_calls                  0
total_day_minutes                0
number_vmail_messages            0
voice_mail_plan                  0
international_plan               0
area_code                        0
churn                            0
dtype: int64
duplicate values in df 0


<h2>Catagorical and Numerical data type in the dataset</h2>

In [17]:
# Get the features with categorical data type.
categorical = list(data.select_dtypes(['object']).columns)
categorical

['state', 'area_code', 'international_plan', 'voice_mail_plan', 'churn']

In [18]:
# Get the features with numerical data type.
numerical = list(data.select_dtypes(['float64','int64']).columns)
numerical

['account_length',
 'number_vmail_messages',
 'total_day_minutes',
 'total_day_calls',
 'total_day_charge',
 'total_eve_minutes',
 'total_eve_calls',
 'total_eve_charge',
 'total_night_minutes',
 'total_night_calls',
 'total_night_charge',
 'total_intl_minutes',
 'total_intl_calls',
 'total_intl_charge',
 'number_customer_service_calls']

In [19]:
# Frequency Encoding for 'state'
frq_dis = data.groupby('state').size()/len(data)
data["state"] = data.state.map(frq_dis)
frq_dis = df_test.groupby('state').size()/len(df_test)
df_test["state"] = df_test.state.map(frq_dis)


In [20]:
# One-hot Encoding for 'area_code'
enc_one = OneHotEncoder(sparse=False)  # Set sparse=False to get a dense array

# Fit and transform the training data
x_train = data["area_code"].values.reshape(-1,1)
val_train = enc_one.fit_transform(x_train)
df_onehot_train = pd.DataFrame(val_train, columns=[f"is_{cat}" for cat in enc_one.categories_[0]])

# Concatenate the one-hot encoded columns with the original data
df_enc_train = pd.concat([df_onehot_train, data.drop("area_code", axis=1)], axis=1)

# Transform the test data using the same encoder
x_test = df_test["area_code"].values.reshape(-1,1)
val_test = enc_one.transform(x_test)
df_onehot_test = pd.DataFrame(val_test, columns=[f"is_{cat}" for cat in enc_one.categories_[0]])

# Concatenate the one-hot encoded columns with the original test data
df_enc_test = pd.concat([df_onehot_test, df_test.drop("area_code", axis=1)], axis=1)

F:\code\installation\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [21]:
df_enc_train.head()

,is_area_code_408,is_area_code_415,is_area_code_510,state,account_length,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,...,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
0,0.0,1.0,0.0,0.022353,107,no,yes,26,161.6,123,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,no
1,0.0,1.0,0.0,0.022588,137,no,no,0,243.4,114,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,no
2,1.0,0.0,0.0,0.022353,84,yes,no,0,299.4,71,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,no
3,0.0,1.0,0.0,0.018353,75,yes,no,0,166.7,113,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,no
4,0.0,0.0,1.0,0.020941,121,no,yes,24,218.2,88,...,108,29.62,212.6,118,9.57,7.5,7,2.03,3,no


In [22]:
df_enc_test.head()

,is_area_code_408,is_area_code_415,is_area_code_510,id,state,account_length,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,...,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
0,0.0,1.0,0.0,1,0.016000,128,no,yes,25,265.1,...,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,0.0,0.0,1.0,2,0.030667,118,yes,no,0,223.4,...,220.6,101,18.75,203.9,118,9.18,6.3,6,1.70,0
2,0.0,1.0,0.0,3,0.009333,62,no,no,0,120.7,...,307.2,76,26.11,203.0,99,9.14,13.1,6,3.54,4
3,0.0,0.0,1.0,4,0.020000,93,no,no,0,190.7,...,218.2,111,18.55,129.6,121,5.83,8.1,3,2.19,3
4,0.0,1.0,0.0,5,0.020000,174,no,no,0,124.3,...,277.1,112,23.55,250.7,115,11.28,15.5,5,4.19,3


In [23]:
# Label Encoding for 'international_plan' and 'voice_mail_plan'
le = LabelEncoder()

# Fit the encoder to the training data
le.fit(df_enc_train['international_plan'])

# Transform both the training and test data using the same encoder
df_enc_train['international_plan'] = le.transform(df_enc_train['international_plan'])
df_enc_test['international_plan'] = le.transform(df_enc_test['international_plan'])

# Fit the encoder to the training data for the second column
le.fit(df_enc_test['voice_mail_plan'])

# Transform both the training and test data using the same encoder
df_enc_train['voice_mail_plan'] = le.transform(df_enc_train['voice_mail_plan'])
df_enc_test['voice_mail_plan'] = le.transform(df_enc_test['voice_mail_plan'])

In [24]:
df_enc_train.head()

,is_area_code_408,is_area_code_415,is_area_code_510,state,account_length,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,...,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
0,0.0,1.0,0.0,0.022353,107,0,1,26,161.6,123,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,no
1,0.0,1.0,0.0,0.022588,137,0,0,0,243.4,114,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,no
2,1.0,0.0,0.0,0.022353,84,1,0,0,299.4,71,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,no
3,0.0,1.0,0.0,0.018353,75,1,0,0,166.7,113,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,no
4,0.0,0.0,1.0,0.020941,121,0,1,24,218.2,88,...,108,29.62,212.6,118,9.57,7.5,7,2.03,3,no


In [25]:
# Normalization
scaler = MinMaxScaler()
numerical_features = df_enc_train.select_dtypes(include=['float64', 'int64']).columns
df_enc_train[numerical_features] = scaler.fit_transform(df_enc_train[numerical_features])
df_enc_test[numerical_features] = scaler.transform(df_enc_test[numerical_features])

In [64]:
df_enc_train.columns

Index(['is_area_code_408', 'is_area_code_415', 'is_area_code_510', 'state',
       'account_length', 'international_plan', 'voice_mail_plan',
       'number_vmail_messages', 'total_day_minutes', 'total_day_calls',
       'total_day_charge', 'total_eve_minutes', 'total_eve_calls',
       'total_eve_charge', 'total_night_minutes', 'total_night_calls',
       'total_night_charge', 'total_intl_minutes', 'total_intl_calls',
       'total_intl_charge', 'number_customer_service_calls', 'churn'],
      dtype='object')

In [65]:
df_enc_test_new= df_enc_test.drop('id', axis=1)

Concluding from the above plots we can actually get rid of total_day_minutes, total_night_minutes, total_eve_minutes and total_intl_minutes to avoid Multicollinearity problem and total_day_calls, total_night_calls, total_eve_calls as they are very less correlated with the dependent feature.

In [66]:
X_train = df_enc_train.drop('churn', axis=1)
y_train = df_enc_train['churn']

X_test = df_enc_test_new

<h1>Model Building with hyperparameter tunning</h1>

In [67]:
# Hyperparameter tuning for different models
def tune_model(model, param_grid):
    grid_search = GridSearchCV(model, param_grid, cv=10, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_

<h1>Logistic Regression</h1>

In [ ]:
#Logistic Regression
# Define the parameter grid

param_grid_lr = {
    'penalty': ['l2', 'elasticnet'],
    'C': [0.1, 1, 10, 100],
    'class_weight': ['balanced', {0: 0.3, 1: 0.7}],
    'solver': ['lbfgs', 'liblinear', 'newton-cg', 'sag', 'saga'],
    'max_iter': [100, 500, 1000]
}

if 'elasticnet' in param_grid_lr['penalty']:
    param_grid_lr['l1_ratio'] = [0, 0.5, 1]
    
# Create the grid search object
best_lr = tune_model(LogisticRegression(), param_grid_lr)

F:\code\installation\Lib\site-packages\sklearn\linear_model\_logistic.py:1171: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
F:\code\installation\Lib\site-packages\sklearn\linear_model\_logistic.py:1171: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
F:\code\installation\Lib\site-packages\sklearn\linear_model\_logistic.py:1171: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
F:\code\installation\Lib\site-packages\sklearn\linear_model\_logistic.py:1171: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
F:\code\installation\Lib\site-packages\sklearn\linear_model\_logistic.py:1171: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
F:\code\installation\Lib\site-packages\sklearn\linear_model\

<h1>Decision Tree Classifier</h1>

In [35]:
# Decision Tree Classifier
param_grid_dt = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_leaf': [1, 2, 4]
}
best_dt = tune_model(DecisionTreeClassifier(), param_grid_dt)
     

<h1>Random Forest</h1>

In [36]:
# Random Forest Classifier
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, 10],
    'min_samples_leaf': [1, 2, 4]
}
best_rf = tune_model(RandomForestClassifier(), param_grid_rf)

<h1>Gradient Boosting Machines (GBM)</h1>

In [ ]:
#Gradient Boosting Machine(GBM)
# Define the parameter grid
param_grid_gb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

# Create the grid search object
best_gb = tune_model(GradientBoostingClassifier(), param_grid=param_grid)

In [33]:
# Evaluate the models
models = {'Decision Tree': best_dt, 'Random Forest': best_rf, 'Logistic Regression': best_lr, 'Gradient Boosting Machines (GBM)': best_gb}
results = {}

In [39]:
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=k_fold, scoring='accuracy')
    mean_score = scores.mean()
    results[name] = mean_score
    print(f"The mean score of cross validation using {name} is {mean_score * 100:.2f}%")

The mean score of cross validation using Decision Tree is 92.05%
The mean score of cross validation using Random Forest is 95.53%
The mean score of cross validation using Logistic Regression is 86.71%
The mean score of cross validation using Gradient Boosting Machines (GBM) is 95.72%


Comparing the results and use the result to predictng the churn

In [41]:
# Predict with the best model (choose the one with the highest score)
best_model_name = max(results, key=results.get)
best_model = models[best_model_name]
best_model.fit(X_train, y_train)

predictions = best_model.predict(X_test)

In [42]:
# Merge the test data ID with the prediction result.
data_test = pd.read_csv('C:\\Users\\Tsi\\Desktop\\10academy\\Week11\\kaggle\\input\\test .csv',low_memory=False)
submission = pd.DataFrame({
        "id": data_test["id"],
        "churn": predictions
    })
submission.head()

,id,churn
0,1,no
1,2,no
2,3,yes
3,4,no
4,5,no


In [44]:
# Save the final result of the test data prediction in CSV format.
submission.to_csv('C:\\Users\\Tsi\\Desktop\\10academy\\Week11\\kaggle\\input\\Prediction_result2.csv', index= None)